# MNIST Image Classification with TensorFlow on Cloud ML Engine

This notebook demonstrates how to implement different image models on MNIST using Estimator/Experiment. It is a companion to <a href="mnist_estimator.ipynb"> mnist_estimator.ipynb </a> which contains the code for the harness.  The Python package itself is in <a href="mnistmodel">mnistmodel</a>

Note the MODEL_TYPE; change it to try out different models

In [1]:
import os
PROJECT = 'cloud-training-demos' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'cloud-training-demos-ml' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1
MODEL_TYPE='dnn'  # 'linear', 'dnn'

# do not change these
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['MODEL_TYPE'] = MODEL_TYPE

In [2]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Model functions

Here are the models

### Linear
A simple low-level matrix multiplication

In [3]:
# linear
def linear_model(img, mode):
  X = tf.reshape(img, [-1, HEIGHT*WIDTH]) # flattened
  #W = tf.Variable(tf.zeros([HEIGHT*WIDTH, NCLASSES]))
  #b = tf.Variable(tf.zeros([NCLASSES]))
  W = tf.Variable(tf.truncated_normal([HEIGHT*WIDTH, NCLASSES], stddev=0.1))
  b = tf.Variable(tf.truncated_normal([NCLASSES], stddev=0.1))
  ylogits = tf.matmul(X, W) + b
  return ylogits, NCLASSES

### Deep Neural Network

A 3-hidden layer network that uses tf.layers to reduce the boilerplate

In [4]:
# dnn
def dnn_model(img, mode):
  X = tf.reshape(img, [-1, HEIGHT*WIDTH]) # flattened
  h1 = tf.layers.dense(X, 300, activation=tf.nn.relu)
  h2 = tf.layers.dense(h1,100, activation=tf.nn.relu)
  h3 = tf.layers.dense(h2, 30, activation=tf.nn.relu)
  ylogits = tf.layers.dense(h3, NCLASSES, activation=None)
  return ylogits, NCLASSES

## Run as a Python module

Let's run it as Python module.  Note the --model parameter

In [ ]:
%bash
rm -rf mnistmodel.tar.gz mnist_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/mnistmodel
python -m trainer.task \
   --output_dir=${PWD}/mnist_trained \
   --train_steps=10 --learning_rate=0.01 --model=$MODEL_TYPE --job-dir=./tmp

Now, let's do it on ML Engine. Note the --model parameter

In [ ]:
%bash
OUTDIR=gs://${BUCKET}/mnist/trained_${MODEL_TYPE}
JOBNAME=mnist_${MODEL_TYPE}_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/mnistmodel/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC_GPU \
   --runtime-version=1.2 \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=100000 --learning_rate=0.01 --model=$MODEL_TYPE

Here are my results:

Model | Accuracy | Time taken | Model description | Run time parameters
--- | :---: | ---
linear | 91.53% | 3 min | | 100 steps, LR=0.01
linear | 92.73% | 8 min | | 1000 steps, LR=0.01
dnn | 98.14% | 9 min | 300-100-30 nodes fully connected | 10000 steps, LR=0.01
dnn | ? | ? | 300-100-30 nodes fully connected | 100000 steps, LR=0.01

## Monitoring training with TensorBoard

Use this cell to launch tensorboard

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/mnist/trained_{}'.format(BUCKET, MODEL_TYPE))

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

## Deploying and predicting with model

Deploy the model:

In [ ]:
%bash
MODEL_NAME="mnist"
MODEL_VERSION=${MODEL_TYPE}
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/mnist/trained_${MODEL_TYPE}/export/Servo | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION}

To predict with the model, let's take one of the example images.

In [ ]:
import json, codecs
import matplotlib.pyplot as plt
IMGNO=5
jsondata = {'image': [mnist.test.images[IMGNO].reshape(HEIGHT, WIDTH).tolist()]}
json.dump(jsondata, codecs.open('test.json', 'w', encoding='utf-8'))
plt.imshow(mnist.test.images[IMGNO].reshape(HEIGHT, WIDTH));

Send it to the prediction service

In [ ]:
%bash
gcloud ml-engine predict --model=mnist --version=${MODEL_TYPE} --json-instances=./test.json

<pre>
# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
</pre>